# C/C++ DLL with  Excel VBA,Python

## 1 C/C++ __stdcall DLL 

  For `Visual Basic` applications (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
VBA can **only** call `__stdcall` functions, not `__cdecl` functions. 
  
*  `__stdcall` creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from right to left)

* ` __declspec(dllexport)` is used on an `exported` function in a DLL

Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

#### Step 1: Create your DLL with `__stdcall` calling convention

Create a DLL with the following code:

* mathfuns.h

* mathfuns.c

Export all functions as `__stdcall`.

In [1]:
%%file ./code/gcc/mathfuns.h

#ifdef BUILD_DLL
#define DLLPORT __declspec(dllexport)
#else
#define DLLPORT __declspec(dllimport)
#endif

DLLPORT __stdcall double fadd(double a,double b);
DLLPORT __stdcall double fmul(double a, double b);

Overwriting ./code/gcc/mathfuns.h


When you create header files for your DLLs, use

* ` __declspec(dllexport) ` adds the `export` directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols

In [2]:
%%file ./code/gcc/mathfuns.c

#include "mathfuns.h"

DLLPORT __stdcall double fadd(double a,double b){
   return (a+b);
}

DLLPORT __stdcall double  fmul(double a, double b){
   return (a*b);
}

Overwriting ./code/gcc/mathfuns.c


In [3]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o ./code/bin/libmathfuns.dll mathfuns.o -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: `Predefine name as a macro`, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an undecorated alias for the `exported function names` that is simply **the name of the function** 

#### Add -static-libgcc ,-output-def=libmathfuns.def

In [4]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o ./code/bin/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=./code/bin/libmathfuns.def

* -static-libgcc

   This option links the GNU `libgcc` library `statically`. 
   

* -output-def=libmathfuns.def

    Name of `.def` file to be created.
    
    **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
%load ./code/bin/libmathfuns.def

In [7]:
%%file ./code/makefile-stdcall

CC=gcc
CFLAGS=-DBUILD_DLL -o

all: libmathfuns.dll

libmathfuns.dll: mathfunsobj
	 $(CC) -shared -o ./code/bin/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def
	 del mathfuns.o
    
mathfunsobj: ./code/gcc/mathfuns.c
	 $(CC) -c $(CFLAGS) mathfuns.o ./code/gcc/mathfuns.c
     
clean:
	 del .\code\bin\libmathfuns.dll

Writing ./code/makefile-stdcall


In [9]:
!make -f ./code/makefile-stdcall

gcc -c -DBUILD_DLL -o mathfuns.o ./code/gcc/mathfuns.c
gcc -shared -o ./code/bin/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def
del mathfuns.o


## 2 Call DLL from Excel VBA(64bits)

Do as the following steps:

**1** `libmathfuns.dll` in the default path of windows'dll `C:\windows\system`


**2** `demo-vba.xlsm` in `./code/gcc/`


**3**  create the `VBA modules` to call the DLL.

press "`ALT+F11`" into **VBA**,then:

* `3.1` create the module **mathfuns** to call library:

```vba
Public Declare PtrSafe Function fadd Lib "libmathfuns" (ByVal a As Double, ByVal b As Double) As Double
Public Declare PtrSafe Function fmul Lib "libmathfuns" (ByVal a As Double, ByVal b As Double) As Double
```
* `3.2` create the module **vbaapi** to call the methods in `mathfuns`:

```vba
Public Function sadd(ByVal a As Double, ByVal b As Double) As Double
    sadd = mathfuns.fadd(a, b)
End Function
 
Public Function smul(ByVal a As Double, ByVal b As Double) As Double
    smul = mathfuns.fmul(a, b)
End Function
```
![vba-code](./img/vba-code.jpg)


**4**  call VBA methods in cells

![demo-vba](./img/demo-vba.jpg)

## 3 call_stdcall DLL from Python

```python
 windll.LoadLibrary
```

In [10]:
%%file ./code/gcc/mathfuns.py

from ctypes import windll,c_double

flib = windll.LoadLibrary('./code/bin/libmathfuns.dll')

def fadd(a,b):
    flib.fadd.argtypes = [c_double,c_double]
    flib.fadd.restype  = c_double
    return flib.fadd(a,b)

def fmul(a,b):
    flib.fmul.argtypes = [c_double,c_double]
    flib.fmul.restype  = c_double
    return flib.fmul(a,b)

Overwriting ./code/gcc/mathfuns.py


#### the python module  `mathfuns.py` is not in the interperte search path

In [11]:
import sys
sys.path.append('./code/gcc')
from mathfuns import *
a=4.6
b=5
print(a,'+',b,'=',fadd(a,b))
print(a,'*',b,'=',fmul(a,b))

4.6 + 5 = 9.6
4.6 * 5 = 23.0


#### SEUIF97：call_stdcall DLL from Python

WINFUNCTYPE

In [12]:
from ctypes import *

flib = windll.LoadLibrary('libseuif97.dll')
prototype = WINFUNCTYPE(c_double, c_double, c_double, c_int)

# ---(p,t) ----------------

def pt(p, t, pid):
    f = prototype(("seupt", flib),)
    result = f(p, t, pid)
    return result


def pt2h(p, t):
    f = prototype(("seupt", flib),)
    result = f(p, t, 4)
    return result

In [13]:
h=pt(16,535,4)
h

3398.1326343129176

## 4 Call __stdcall DLL from C 

In [14]:
%%file ./code/gcc/mainmathfuns.c

#include <stdio.h>
#include "mathfuns.h"
 
int main() {
    double a=5.3;
    double b=6.1;
    double radd=fadd(a,b);
    double rmul=fmul(a,b);
    printf("%.3f + %.3f = %.3f \n",a,b,radd);
    printf("%.3f * %.3f = %.3f \n",a,b,rmul);
    return 0;
}

Overwriting ./code/gcc/mainmathfuns.c


In [15]:
!gcc -c -o mainmathfuns.o ./code/gcc/mainmathfuns.c 
!gcc -o  ./code/bin/mainmathfuns.exe mainmathfuns.o -I./code/gcc/ -L./code/bin/ -lmathfuns

In [16]:
!.\code\bin\mainmathfuns

5.300 + 6.100 = 11.400 
5.300 * 6.100 = 32.330 


#### Building with makefile

In [20]:
%%file ./code/makefile-stdcall-c

all: mainmathfuns.exe

clean:
	del .\code\bin\mainmathfuns.exe

mainmathfuns.exe: mainmathfuns.o ./code/gcc/mathfuns.h 
	gcc -o ./code/bin/mainmathfuns mainmathfuns.o -I./code/gcc -L./code/bin/ -lmathfuns
	del *.o

mainmathfuns.o: ./code/gcc/mainmathfuns.c 
	gcc -c ./code/gcc/mainmathfuns.c 

Overwriting ./code/makefile-stdcall-c


In [21]:
!make -f ./code/makefile-stdcall-c

gcc -o ./code/bin/mainmathfuns mainmathfuns.o -I./code/gcc -L./code/bin/ -lmathfuns
del *.o


In [22]:
!.\code\bin\mainmathfuns

5.300 + 6.100 = 11.400 
5.300 * 6.100 = 32.330 


### Further reading

* ctypes http://docs.python.org/3/library/ctypes.html


* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx


* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx


* Getting Started with VBA in Office 2010: https://msdn.microsoft.com/library/office/ee814735(v=office.14)

   
* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html


* **Excel4Engineering** https://github.com/PySEE/Excel4Engineering